<a href="https://colab.research.google.com/github/Ryo-Tanohata/sig_beginner9/blob/main/beginner9_compares2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://signate.jp/competitions/434#abstract

#1 データの取得
自分の環境に合わせてデータを取得して読み込んで下さい。

In [1]:
#signate_begginer9.zipをGoogle Drive からひっぱってくるやつ
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth
 
auth.authenticate_user()
 
drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'signate_begginer9.zip'", fields="files(id)").execute()
signate_api_key = results.get('files', [])
 
filename = "/content/input/signate_begginer9.zip"
os.makedirs(os.path.dirname(filename), exist_ok=True)
 
request = drive_service.files().get_media(fileId=signate_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


#2 ファイルの解凍

In [2]:
import os
path="./input"
zip_files=os.listdir(path)
zip_files

['signate_begginer9.zip']

In [3]:
import zipfile
for i in zip_files:
    if ".zip" in i:
        with zipfile.ZipFile("input/"+i) as existing_zip:
            print(f'open {i}')
            existing_zip.extractall('input')
    else:
        continue

open signate_begginer9.zip


# ライブラリのインストール

In [4]:
import warnings
import gc
import pandas as pd
import pandas_profiling as pdp
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, precision_recall_curve, roc_curve, average_precision_score
warnings.simplefilter('ignore')

In [5]:
import pandas as pd
import numpy as np
import random as rnd
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble  import HistGradientBoostingClassifier
#from sklearn.ensemble  import HistGradientBoostingRegressor
from lightgbm import LGBMClassifier

# データの前処理

In [6]:
 # 読み込むデータが格納されたディレクトリのパス，必要に応じて変更の必要あり
path = "/content/input/"

train_df = pd.read_csv(path + "train.csv")
test_df = pd.read_csv(path + "test.csv")
submit = pd.read_csv(path + "sample_submission.csv", header=None)

# 訓練データ、テストデータがわかるようにダミーの目的変数を代入
test_df['target']=-999

# 訓練データ、テストデータを結合
all_df = pd.concat([train_df,test_df])
del train_df,test_df
gc.collect()

# def eGFR(x):
#     if x['sex']==1:
#       return 194*(x['serum_creatinine']**-1.094)*(x['age']**-0.287)
#     else:
#       return 194*(x['serum_creatinine']**-1.094)*(x['age']**-0.287)*0.739

# all_df["eGFR"] = all_df.apply(eGFR, axis=1)

# 訓練データ、テストデータの分割
train_df = all_df[all_df['target']!=-999]
test_df = all_df[all_df['target']==-999]

y=train_df['target']
X=train_df.drop(['target','id'], axis=1)
test=test_df.drop(['target','id'], axis=1)

In [7]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

In [8]:
import time
from contextlib import contextmanager

@contextmanager
def timer(name):
    t0 = time.time()
    yield
    print(f'[{name}] done in {time.time() - t0:.0f} s')

In [10]:
pip install -q optuna

     |████████████████████████████████| 296kB 5.5MB/s 
     |████████████████████████████████| 81kB 5.2MB/s 
     |████████████████████████████████| 163kB 7.5MB/s 
     |████████████████████████████████| 143kB 7.9MB/s 
     |████████████████████████████████| 112kB 9.3MB/s 
     |████████████████████████████████| 51kB 4.6MB/s 
     |████████████████████████████████| 81kB 5.7MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-cp37-none-any.whl size=11107 sha256=83816bdf58701c61276789926431c44b318d0b8e9498b5ea19c902ce88ae55d3
  Stored in directory: /root/.cache/pip/wheels/25/af/b8/3407109267803f4015e1ee2ff23be0c8c19ce4008665931ee1
Successfully built pyperclip


In [11]:
import optuna
import lightgbm as lgbm
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score, make_scorer

In [12]:
SEED = 0
NFOLDS = 5
NTRIAL = 1000

LR = 0.1
ITER = 100

In [13]:
def tuning_parameter(trial, classifier):
    if classifier == 'LightGBM':
        params = {}
        params['objective'] = 'binary'
        params['random_state'] = SEED
        params['metric'] = 'binary_logloss'
        params['verbosity'] = -1
        params['boosting_type'] = trial.suggest_categorical('boosting', ['gbdt', 'dart', 'goss'])

        # モデル訓練のスピードを上げる
        #params['bagging_freq'] = trial.suggest_int('max_bins', 0, 5)
        params['save_binary'] = False

        # 推測精度を向上させる
        params['learning_rate'] = LR
        params['num_iterations'] = ITER
        params['num_leaves'] = trial.suggest_int('num_leaves', 5, 100)
        params['max_bins'] = trial.suggest_int('max_bins', 2, 256)

        # 過学習対策
        # early stoppingは今回使わない。切り方によって、性能を高く見積もる可能性があるため。
        # データ数が少ないため、早期に切り上げる必要性を感じないため。
        #params['eary_stopping_round']
        params['min_data_in_leaf'] = trial.suggest_int('min_data_in_leaf', 1, 100)
        params['feature_fraction'] = trial.suggest_uniform('top_rate', 0.0, 1.0)
        #params['bagging_fraction'] = trial.suggest_uniform('bagging_fraction', 0, 1.0)
        params['min_child_weight'] = trial.suggest_int('min_child_weight', 0, 1e-3)
        params['lambda_l1'] = trial.suggest_int('lambda_l1', 0, 500)
        params['lambda_l2'] =  trial.suggest_int('lambda_l2', 0, 500)
        params['min_gain_to_split'] = 0
        params['max_depth'] = trial.suggest_int('max_depth', 6, 10)

        if params['boosting_type'] == 'dart':
            params['drop_rate'] = trial.suggest_loguniform('drop_rate', 1e-8, 1.0)
            params['skip_drop'] = trial.suggest_loguniform('skip_drop', 1e-8, 1.0)
        if params['boosting_type'] == 'goss':
            params['top_rate'] = trial.suggest_uniform('top_rate', 0.0, 1.0)
            params['other_rate'] = trial.suggest_uniform('other_rate', 0.0, 1.0 - params['top_rate'])
        return params

    if classifier == 'HistGradientBoostingClassifier':
        params = {}

        params['random_state'] = SEED
        params['loss'] = 'binary_crossentropy'
        params['verbose'] = -1

        # モデル訓練のスピードを上げる
        params['tol'] = trial.suggest_loguniform('tol', 1e-8, 1e-1)

        # 推測精度を向上させる
        params['learning_rate'] = LR
        params['max_iter'] = ITER
        params['max_leaf_nodes'] = trial.suggest_int('max_leaf_nodes', 5, 100)
        params['max_bins'] = trial.suggest_int('max_bins', 2, 256)
        params['min_samples_leaf'] = trial.suggest_int('min_samples_leaf', 1, 100)

        # 過学習対策
        #params['n_iter_no_change']
        #iparams['scoring'] 
        params['max_depth'] = trial.suggest_int('max_depth', 6, 10)
        params['validation_fraction'] =  0.1 #trial.suggest_uniform('top_rate', 0.0, 1.0)
        params['l2_regularization'] = trial.suggest_int('l2_regularization', 0, 500)
        return params

In [14]:
def estimator(classifier, params):
    if classifier == 'LightGBM': return lgbm.LGBMClassifier(**params)
    if classifier == 'HistGradientBoostingClassifier': return HistGradientBoostingClassifier(**params)

In [15]:
def evaluate_score():
    return  {
        'accuracy': make_scorer(accuracy_score)
    }

In [19]:
class Objective(object):
    def __init__(self, dataset):
        self.X, self.y = dataset['training'], dataset['answer']

    def __call__(self, trial):

        classifier = dataset['classifier'] if 'classifier' in dataset else trial.suggest_categorical('classifier', ['LightGBM', 'HistGradientBoostingClassifier'])

        params = tuning_parameter(trial, classifier)
        clf = estimator(classifier, params)
        score = evaluate_score()

        kf = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=SEED)
        scores = cross_validate(estimator=clf, X=self.X, y=self.y, cv=kf, scoring=score, n_jobs=-1)

        return scores['test_accuracy'].mean()

In [17]:
def bayesian_optimize_parameter(dataset):
    objective = Objective(dataset)
    study = optuna.create_study(
        pruner=optuna.pruners.SuccessiveHalvingPruner(min_resource = 1, reduction_factor = 4, min_early_stopping_rate = 0)
        #, direction='maximize')
    )
    study.optimize(objective, n_trials=NTRIAL)
    return study.trials_dataframe(), study.best_params, study.best_value

In [20]:
dataset = {'classifier': 'HistGradientBoostingClassifier', 'training': X, 'answer': y}

with timer('HistGradientBoostingClassifier'):
    dataframe, params, value = bayesian_optimize_parameter(dataset)
    dataframe.drop(['number', 'datetime_start', 'datetime_complete'], axis=1).sort_values(['value'], ascending=True)[:20]
    print(f'[params]: \n{params}')
    print(f'[value]: {value}')

[I 2021-04-26 22:29:01,465] A new study created in memory with name: no-name-842e400a-287b-4807-913f-5eea7a54eaeb
[I 2021-04-26 22:29:02,890] Trial 0 finished with value: 0.8020000000000002 and parameters: {'tol': 1.288909020174667e-07, 'max_leaf_nodes': 35, 'max_bins': 98, 'min_samples_leaf': 41, 'max_depth': 9, 'l2_regularization': 434}. Best is trial 0 with value: 0.8020000000000002.
[I 2021-04-26 22:29:03,323] Trial 1 finished with value: 0.8020000000000002 and parameters: {'tol': 8.032518978415196e-05, 'max_leaf_nodes': 83, 'max_bins': 17, 'min_samples_leaf': 14, 'max_depth': 7, 'l2_regularization': 346}. Best is trial 0 with value: 0.8020000000000002.
[I 2021-04-26 22:29:03,659] Trial 2 finished with value: 0.8020000000000002 and parameters: {'tol': 0.014818255822371723, 'max_leaf_nodes': 8, 'max_bins': 212, 'min_samples_leaf': 38, 'max_depth': 9, 'l2_regularization': 474}. Best is trial 0 with value: 0.8020000000000002.
[I 2021-04-26 22:29:04,105] Trial 3 finished with value: 0.

[params]: 
{'tol': 0.0007173339366954107, 'max_leaf_nodes': 87, 'max_bins': 240, 'min_samples_leaf': 95, 'max_depth': 7, 'l2_regularization': 119}
[value]: 0.797
[HistGradientBoostingClassifier] done in 268 s


In [22]:
dataset = {'classifier': 'LightGBM', 'training': X, 'answer': y}

with timer('LightGBM'):
    dataframe, params, value = bayesian_optimize_parameter(dataset)
    dataframe.drop(['number', 'datetime_start', 'datetime_complete'], axis=1).sort_values(['value'], ascending=True)[:20]
    print(f'[params]: \n{params}')
    print(f'[value]: {value}')

[I 2021-04-26 22:38:49,710] A new study created in memory with name: no-name-53e2b875-5787-4999-bf64-fbfe8d9ddcb8
[I 2021-04-26 22:38:49,782] Trial 0 finished with value: 0.8020000000000002 and parameters: {'boosting': 'dart', 'num_leaves': 70, 'max_bins': 124, 'min_data_in_leaf': 86, 'top_rate': 0.7535299385435651, 'min_child_weight': 0, 'lambda_l1': 56, 'lambda_l2': 195, 'max_depth': 8, 'drop_rate': 3.222071365144569e-05, 'skip_drop': 1.0334916436474538e-06}. Best is trial 0 with value: 0.8020000000000002.
[I 2021-04-26 22:38:49,841] Trial 1 finished with value: 0.8020000000000002 and parameters: {'boosting': 'gbdt', 'num_leaves': 58, 'max_bins': 41, 'min_data_in_leaf': 14, 'top_rate': 0.305934475370954, 'min_child_weight': 0, 'lambda_l1': 127, 'lambda_l2': 480, 'max_depth': 6}. Best is trial 0 with value: 0.8020000000000002.
[I 2021-04-26 22:38:49,903] Trial 2 finished with value: 0.8020000000000002 and parameters: {'boosting': 'goss', 'num_leaves': 87, 'max_bins': 9, 'min_data_in_l

[params]: 
{'boosting': 'dart', 'num_leaves': 84, 'max_bins': 226, 'min_data_in_leaf': 77, 'top_rate': 0.527219234202471, 'min_child_weight': 0, 'lambda_l1': 41, 'lambda_l2': 199, 'max_depth': 9, 'drop_rate': 0.01683629558061682, 'skip_drop': 0.007865376876215645}
[value]: 0.19800000000000004
[LightGBM] done in 148 s


## 交差検証の準備

In [ ]:
SVM = SVC()
RF = RandomForestClassifier()
LR = LogisticRegression()
HGBC = HistGradientBoostingClassifier()
LGB= LGBMClassifier()

scores = []
modelnames = ['HistGradient','SVM','RandumForest','LightGBM','LogisticRegression']
models = [HGBC,SVM,RF,LGB,LR]

for i in models:
    score = cross_val_score(i, X, y, scoring = 'accuracy' , cv = 5).mean()
    scores.append(score)

pd.DataFrame(scores, index=modelnames,
            columns=['CV Scores']).sort_values(by = 'CV Scores', ascending = False)    

,CV Scores
RandumForest,0.852
HistGradient,0.850
LightGBM,0.850
SVM,0.802
LogisticRegression,0.799


In [ ]:
folds = 5
kf = StratifiedKFold(n_splits=folds,shuffle=True,random_state=0)

In [ ]:
feats=list(data.columns)
feats

['age',
 'anaemia',
 'creatinine_phosphokinase',
 'diabetes',
 'ejection_fraction',
 'high_blood_pressure',
 'platelets',
 'serum_creatinine',
 'serum_sodium',
 'sex',
 'smoking',
 'time']

# 学習・予測

In [ ]:
preds={} #アンサンブル前の予測

## HistGradientBoostingClassifier

### Kfold

In [ ]:
y_cv_train_pred=y.copy()
y_cv_pred=y.copy()
y_test_list=[]
for i,(train_index,valid_index) in enumerate(kf.split(X, y)):
    X_cv_train = X.iloc[train_index]
    X_cv_valid = X.iloc[valid_index]
    y_cv_train = y.iloc[train_index]
    y_cv_valid = y.iloc[valid_index]
 
    model = HistGradientBoostingClassifier(loss='binary_crossentropy',max_iter=10001,learning_rate=0.1,random_state=0)
    model.fit(X_cv_train,y_cv_train)
    y_cv_train_pred[train_index] = model.predict(X_cv_train)
    y_cv_pred[valid_index] = model.predict(X_cv_valid) #train時のデータに合わせてpredを入れる #本コンペの評価はaccuracyのため確率のみ算出
    y_test_preds = model.predict_proba(test)
    y_test_list.append(y_test_preds)
    print(f'Fold {i+1}, ACC : {accuracy_score(y_cv_train, y_cv_train_pred[train_index]):.6f}, val_ACC : {accuracy_score(y_cv_valid, y_cv_pred[valid_index]):.6f}')
    #y_cv_predproba_list.append(model.predict_proba(test_))
 
print('Full ACC score %.6f' % accuracy_score(y, y_cv_pred))

Fold 1, ACC : 1.000000, val_ACC : 0.870000
Fold 2, ACC : 1.000000, val_ACC : 0.830000
Fold 3, ACC : 1.000000, val_ACC : 0.870000
Fold 4, ACC : 1.000000, val_ACC : 0.890000
Fold 5, ACC : 1.000000, val_ACC : 0.855000
Full ACC score 0.863000


In [ ]:
pred_proba_hist=0
for i in y_test_list:
    pred_proba_hist += i
pred_proba=pred_proba_hist/len(y_test_list)
pred={"hist": pred_proba} #1000個の推論データ
preds.update(pred)
#test_pred=(pred_proba_hist/len(y_test_list)).argmax(axis=1)
#test_pred

## LightGBMClassifier

In [ ]:
import lightgbm as lgb
y_cv_train_pred=y.copy()
y_cv_pred=y.copy()
y_test_list=[]
for i,(train_index,valid_index) in enumerate(kf.split(X, y)):
    X_cv_train = X.iloc[train_index]
    X_cv_valid = X.iloc[valid_index]
    y_cv_train = y.iloc[train_index]
    y_cv_valid = y.iloc[valid_index]
 
    lgb_train = lgb.Dataset(X_cv_train, y_cv_train,)
    lgb_eval = lgb.Dataset(X_cv_valid, y_cv_valid, reference=lgb_train,)

    model = lgb.LGBMClassifier(loss='binary_crossentropy',max_iter=10001,learning_rate=0.1,random_state=0)
    model.fit(X_cv_train, y_cv_train, eval_set=[(X_cv_valid, y_cv_valid)],
              verbose=False, eval_metric='accuracy')#, categorical_feature=categorical_features) # モデルの学習
    y_cv_train_pred[train_index] = model.predict(X_cv_train)
    y_cv_pred[valid_index] = model.predict(X_cv_valid) #train時のデータに合わせてpredを入れる #本コンペの評価はaccuracyのため確率のみ算出
    y_test_preds = model.predict_proba(test)
    y_test_list.append(y_test_preds)
    print(f'Fold {i+1}, ACC : {accuracy_score(y_cv_train, y_cv_train_pred[train_index]):.6f}, val_ACC : {accuracy_score(y_cv_valid, y_cv_pred[valid_index]):.6f}')
    #y_cv_predproba_list.append(model.predict_proba(test_))
 
print('Full ACC score %.6f' % accuracy_score(y, y_cv_pred))

Fold 1, ACC : 1.000000, val_ACC : 0.870000
Fold 2, ACC : 1.000000, val_ACC : 0.835000
Fold 3, ACC : 1.000000, val_ACC : 0.860000
Fold 4, ACC : 1.000000, val_ACC : 0.885000
Fold 5, ACC : 1.000000, val_ACC : 0.835000
Full ACC score 0.857000


In [ ]:
pred_proba_gbm=0
for i in y_test_list:
    pred_proba_gbm += i
pred_proba=pred_proba_gbm/len(y_test_list)
pred={"lgbm": pred_proba} #1000個の推論データ
preds.update(pred)

In [ ]:
sub_pred_proba=(preds['hist']+preds['lgbm'])/2
sub_pred=sub_pred_proba.argmax(axis=1)
sub_pred

In [ ]:
submit[1] = sub_pred
submit.to_csv(path + "submission.csv", index=False, header=False)

In [ ]:
pd.read_csv(path + "submission.csv", header=None)

,0,1
0,1,0
1,2,1
2,4,0
3,5,0
4,6,0
...,...,...
995,1992,0
996,1993,0
997,1995,0
998,1997,0
